# **YOLOv5-StrongSORT-GSI**

Реализована модель YOLOv5-StrongSORT-GSI
*   Модель детектирования YOLOv5
*   Алгоритм трекинга StrongSORT
*   Метод постобработки GSI (линейная интерполяция и гауссовское сглаживание)



In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
import shutil
import numpy as np

from PIL import Image
from os.path import join
from pathlib import Path
from collections import defaultdict

import torch

from sklearn.gaussian_process.kernels import RBF, Matern, ExpSineSquared, ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor as GPR

# **VisDrone to YOLO**

In [ ]:
def rename_images(path):
    for folder in os.listdir(path):
        for image in os.listdir(os.path.join(path, folder)):
            temp = os.path.join(path, folder)
            os.rename(os.path.join(temp, image), temp + ' ' + image)
        shutil.rmtree(os.path.join(path, folder))

In [ ]:
def get_label(path):
    with open(path, 'r') as f:
        label = [line.strip('\n').split(',') for line in f.readlines()]
    return label

In [ ]:
def split_label_by_frames(label, frames_num):
    labels = [[] for _ in range(frames_num)]
    for i in range(len(label)):
        labels[int(label[i][0]) - 1].append(label[i])
    return labels

In [ ]:
def convert_box(size, box):
    dw = 1. / size[1]
    dh = 1. / size[0]
    return [str((int(box[0]) + int(box[2]) / 2) * dw), 
            str((int(box[1]) + int(box[3]) / 2) * dh), 
            str(int(box[2]) * dw), 
            str(int(box[3]) * dh)]

In [ ]:
def visdrone_to_yolo(labels, sizes):
    new_labels = [[[labels[i][j][k] for k in range(len(labels[i][j]))
                    if k != 0 and k != 1 and k != 6 and k != 8 and k != 9]
                   for j in range(len(labels[i]))] for i in range(len(labels))]
    
    yolo_labels = [[[new_labels[i][j][4]] for j in range(len(new_labels[i]))] for i in range(len(new_labels))]
    
    for i in range(len(new_labels)):
        for j in range(len(new_labels[i])):
            yolo_labels[i][j].extend(convert_box(sizes[i], new_labels[i][j]))

    return yolo_labels

In [ ]:
def labels_to_txt(right_labels, file, labels_path, images_path):
    os.remove(os.path.join(labels_path, file))
    images = [image for image in os.listdir(images_path) if image.startswith(file.rstrip('.txt'))]

    for i in range(len(right_labels)):
        temp = os.path.join(labels_path, images[i].rstrip('.jpg'))
        with open(os.path.join(temp, '.txt'), 'w') as f:
            for j in range(len(right_labels[i])):
                f.write(' '.join(right_labels[i][j]))
                if j != len(right_labels[i]) - 1:
                    f.write('\n')

**train**

In [ ]:
train_images_path = '/content/drive/MyDrive/dataset/train/images'
train_labels_path = '/content/drive/MyDrive/dataset/train/labels'

old_train_images_path = '/content/drive/MyDrive/VisDrone2019-MOT-train/sequences'
old_train_labels_path = '/content/drive/MyDrive/VisDrone2019-MOT-train/annotations'

In [ ]:
shutil.copytree(old_train_images_path, train_images_path)

In [ ]:
shutil.copytree(old_train_labels_path, train_labels_path)

In [ ]:
rename_images(train_images_path)

**val**

In [ ]:
val_images_path = '/content/drive/MyDrive/dataset/val/images'
val_labels_path = '/content/drive/MyDrive/dataset/val/labels'

old_val_images_path = '/content/drive/MyDrive/VisDrone2019-MOT-val/sequences'
old_val_labels_path = '/content/drive/MyDrive/VisDrone2019-MOT-val/annotations'

In [ ]:
shutil.copytree(old_val_images_path, val_images_path)

In [ ]:
shutil.copytree(old_val_labels_path, val_labels_path)

In [ ]:
rename_images(val_images_path)

**test**

In [ ]:
test_images_path = '/content/drive/MyDrive/dataset/test/images'
test_labels_path = '/content/drive/MyDrive/dataset/test/labels'

old_test_images_path = '/content/drive/MyDrive/VisDrone2019-MOT-test/sequences'
old_test_labels_path = '/content/drive/MyDrive/VisDrone2019-MOT-test/annotations'

In [ ]:
shutil.copytree(old_test_images_path, test_images_path)

In [ ]:
shutil.copytree(old_test_labels_path, test_labels_path)

In [ ]:
rename_images(test_images_path)

**visdrone labels to yolo labels**

In [ ]:
path1 = train_labels_path
path2 = train_images_path

for label_file in os.listdir(path1):
    
    label = get_label(path1 + '/' + label_file)
    labels = split_label_by_frames(label, len([name for name in os.listdir(path2)
                                               if name.startswith(label_file.rstrip('.txt'))]))
    images_sizes = [Image.open(path2 + '/' + i).size for i in os.listdir(path2)
                        if i.startswith(label_file.rstrip('.txt'))]
    
    right_labels = visdrone_to_yolo(labels, images_sizes)
    labels_to_txt(right_labels, label_file, path1, path2)

In [ ]:
path1 = val_labels_path
path2 = val_images_path

for label_file in os.listdir(path1):
    
    label = get_label(os.path.join(path1, label_file))
        
    labels = split_label_by_frames(label, len([name for name in os.listdir(path2) if name.startswith(label_file.rstrip('.txt'))]))
    
    images_sizes = [Image.open(os.path.join(path2, i)).size for i in os.listdir(path2) if i.startswith(label_file.rstrip('.txt'))]
    
    right_labels = visdrone_to_yolo(labels, images_sizes)
    labels_to_txt(right_labels, label_file, path1, path2)

In [ ]:
path1 = test_labels_path
path2 = test_images_path

for label_file in os.listdir(path1):
    
    label = get_label(path1 + '/' + label_file)
    labels = split_label_by_frames(label, len([name for name in os.listdir(path2)
                                               if name.startswith(label_file.rstrip('.txt'))]))
    images_sizes = [Image.open(path2 + '/' + i).size for i in os.listdir(path2)
                        if i.startswith(label_file.rstrip('.txt'))]
    
    right_labels = visdrone_to_yolo(labels, images_sizes)
    labels_to_txt(right_labels, label_file, path1, path2)

In [ ]:
path = '/content/drive/MyDrive/dataset/val/labels'

def get_label(path):
    with open(path, 'r') as f:
        label = [line.strip('\n').split(' ') for line in f.readlines()]
    return label


def change_label(label):

    new_label = [label[i] for i in range(len(label)) if label[i][0] != '0' and label[i][0] != '11']

    for i in range(len(new_label)):
        if new_label[i][0] == '1':
            new_label[i][0] = '0'
        elif new_label[i][0] == '2':
            new_label[i][0] = '1'
        elif new_label[i][0] == '3':
            new_label[i][0] = '2'
        elif new_label[i][0] == '4':
            new_label[i][0] = '3'
        elif new_label[i][0] == '5':
            new_label[i][0] = '4'
        elif new_label[i][0] == '6':
            new_label[i][0] = '5'
        elif new_label[i][0] == '7':
            new_label[i][0] = '6'
        elif new_label[i][0] == '8':
            new_label[i][0] = '7'
        elif new_label[i][0] == '9':
            new_label[i][0] = '8'
        elif new_label[i][0] == '10':
            new_label[i][0] = '9'

    return new_label


def print_new_label(new_label, path):
    with open(path, 'w') as f:
        for i in range(len(new_label)):
            for j in range(len(new_label[i])):
                f.write(new_label[i][j])
                if j != len(new_label[i]):
                    f.write(' ')
            if i != len(new_label):
                f.write('\n')


for i in os.listdir(path):
        label = get_label(os.path.join(path, i))
        new_label = change_label(label)
        print_new_label(new_label, os.path.join(path, i))

In [ ]:
! python strong_sort.py MOT20 test --NSA --EMA --MC --woC --AFLink --GSI

# **Video From Images**

In [ ]:
image_folder='/content/drive/MyDrive/images'
fps=20

image_files = [os.path.join(image_folder,img)
               for img in os.listdir(image_folder)
               if img.endswith(".jpg")]

clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('test.mp4')

# **YOLOv5 + StrongSORT**

In [ ]:
%cd /content/drive/MyDrive/yolov5
! pip install -r requirements.txt

In [ ]:
! python val.py --batch-size 32 --weights /content/drive/MyDrive/weights/yolov5x-40-epochs-VisDrone-upd.pt --data /content/drive/MyDrive/dataset/dataset.yaml

In [ ]:
%cd /content/drive/MyDrive
%cd tph-yolov5
! pip install -r requirements.txt

In [ ]:
! pip install torchvision==0.10.1
! pip install torch==1.9.1

In [ ]:
! python val.py --weights /content/drive/MyDrive/weights/yolov5l-xs-1.pt --data /content/drive/MyDrive/dataset/dataset.yaml --augment --save-txt  --save-conf --task val --batch-size 64 --verbose --name v5l-tph-plus

In [ ]:
%cd /content/drive/MyDrive/yolov8_tracking
! pip install -r requirements.txt

In [ ]:
from yolov5.utils.dataloaders import LoadImages
from yolov5.utils.plots import Annotator, colors, save_one_box
from yolov5.utils.general import (LOGGER, Profile, check_img_size, non_max_suppression, scale_boxes, check_requirements, cv2,
                                  check_imshow, xyxy2xywh, increment_path, strip_optimizer, colorstr, print_args, check_file)

In [ ]:
! python track.py --source /content/drive/MyDrive/dataset/val/sequences/uav0000117_02622_v --tracking-method strongsort --yolo-weights /content/drive/MyDrive/weights/yolov5imprv-40epochs-VisDrone.pt --reid-weights osnet_x0_25_market1501.pt --save-txt --save-vid

# **GSI**

In [ ]:
def LinearInterpolation(input_, interval):
    input_ = input_[np.lexsort([input_[:, 0], input_[:, 1]])]
    output_ = input_.copy()
    id_pre, f_pre, row_pre = -1, -1, np.zeros((10,))
    for row in input_:
        f_curr, id_curr = row[:2].astype(int)
        if id_curr == id_pre:
            if f_pre + 1 < f_curr < f_pre + interval:
                for i, f in enumerate(range(f_pre + 1, f_curr), start=1):
                    step = (row - row_pre) / (f_curr - f_pre) * i
                    row_new = row_pre + step
                    output_ = np.append(output_, row_new[np.newaxis, :], axis=0)
        else:
            id_pre = id_curr
        row_pre = row
        f_pre = f_curr
    output_ = output_[np.lexsort([output_[:, 0], output_[:, 1]])]
    return output_


def GaussianSmooth(input_, tau):
    output_ = []
    ids = set(input_[:, 1])
    for id_ in ids:
        tracks = input_[input_[:, 1] == id_]
        len_scale = np.clip(tau * np.log(tau ** 3 / len(tracks)), tau ** -1, tau ** 2)
        gpr = GPR(RBF(len_scale, 'fixed'))
        t = tracks[:, 0].reshape(-1, 1)
        x = tracks[:, 2].reshape(-1, 1)
        y = tracks[:, 3].reshape(-1, 1)
        w = tracks[:, 4].reshape(-1, 1)
        h = tracks[:, 5].reshape(-1, 1)

        gpr.fit(t, x)
        xx = gpr.predict(t)
        gpr.fit(t, y)
        yy = gpr.predict(t)
        gpr.fit(t, w)
        ww = gpr.predict(t)
        gpr.fit(t, h)
        hh = gpr.predict(t)
        output_.extend([
            [t[i, 0], id_, xx[i], yy[i], ww[i], hh[i], 1, -1, -1 , -1, tracks[i, -1]] for i in range(len(t))
        ])
    return np.array(output_)


def GSInterpolation(path_in, path_out, interval, tau):
    input_ = np.loadtxt(path_in)
    li = LinearInterpolation(input_, interval)
    gsi = GaussianSmooth(li, tau)
    gsi = gsi[np.lexsort([gsi[:, 1], gsi[:, 0]])]
    np.savetxt(path_out.rstrip('.txt') + '-gsi.txt', gsi, fmt='%d %d %.2f %.2f %.2f %.2f %.2f %d %d %d %d')

In [ ]:
path_save = '/content/drive/MyDrive/yolov8_tracking/runs/track/exp28/tracks/uav0000117_02622_v.txt'

GSInterpolation(path_in=path_save, path_out=path_save, interval=5, tau=9)

In [ ]:
path = '/content/drive/MyDrive/yolov8_tracking/runs/track/exp28/imgs'

if os.path.exists(path):
    shutil.rmtree(path)

if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
source = '/content/drive/MyDrive/dataset/val/sequences/uav0000117_02622_v'
gsi_path = '/content/drive/MyDrive/yolov8_tracking/runs/track/exp28/tracks/uav0000117_02622_v-gsi.txt'
gsi_np = np.loadtxt(gsi_path)

In [ ]:
dataset = LoadImages(source)
names = ['pedestrian', 'people', 'bicycle', 'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor']

In [ ]:
i = 0
img_path = '/content/drive/MyDrive/yolov8_tracking/runs/track/exp28/imgs'
for frame_idx, (path, im, im0s, vid_cap, s) in enumerate(dataset):
    im0 = im0s.copy()
    annotator = Annotator(im0, line_width=2, example=str(names))
    for gsi_np_obj in gsi_np:
        if gsi_np_obj[0] == frame_idx:
            bboxes = gsi_np_obj[2:6]
            bboxes[2] = bboxes[2] + bboxes[0]
            bboxes[3] = bboxes[3] + bboxes[1]
            id = gsi_np_obj[1]
            cls = gsi_np_obj[-1]
            c = int(cls)
            id = int(id)
            label = (f'{id} {names[c]}')
            color = colors(c, True)
            annotator.box_label(bboxes, label, color=color)
    im0 = annotator.result()
            
    cv2.imwrite(img_path + '/' + str(i) + '.jpg', im0)
    i += 1

# **The End**